In [1]:
import string
from collections import defaultdict
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models

c:\users\gyorgym\apps\miniconda3\envs\dl\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
tweets_csv = pd.read_csv("Data/twcs.csv")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GyorgyM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\GyorgyM\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
first_mask = tweets_csv.in_response_to_tweet_id.isnull()
first_tweets = tweets_csv.loc[first_mask, ["tweet_id", "text"]].copy()
response_tweets = tweets_csv.loc[~first_mask, ["tweet_id", "text", "in_response_to_tweet_id"]].copy()
response_tweets["in_response_to_tweet_id"] = response_tweets["in_response_to_tweet_id"].astype("int64").values
response_tweets.head()

,tweet_id,text,in_response_to_tweet_id
0,1,@115712 I understand. I would like to assist y...,3
1,2,@sprintcare and how do you propose we do that,1
2,3,@sprintcare I have sent several private messag...,4
3,4,@115712 Please send us a Private Message so th...,5
4,5,@sprintcare I did.,6


In [32]:
left = first_tweets
right = response_tweets

merged = pd.merge(left, right, how="left", left_on="tweet_id", right_on="in_response_to_tweet_id", 
                  suffixes=("", "_1"))
# merged = merged[[col for col in merged.columns if col != "in_response_to_tweet_id"]]
merged.head()

,tweet_id,text,tweet_id_1,text_1,in_response_to_tweet_id
0,8,@sprintcare is the worst customer service,6,@115712 Can you please send us a private messa...,8
1,8,@sprintcare is the worst customer service,9,@115712 I would love the chance to review the ...,8
2,8,@sprintcare is the worst customer service,10,@115712 Hello! We never like our customers to ...,8
3,18,@115714 y’all lie about your “great” connectio...,17,@115713 H there! We'd definitely like to work ...,18
4,20,"@115714 whenever I contact customer support, t...",19,@115715 Please send me a private message so th...,20


In [ ]:
left = first_tweets
right = response_tweets

n = 1
while not right.empty:
    left_on = "tweet_id_" + str(n - 1) if n > 1 else "tweet_id"
    right_on = "in_response_to_tweet_id"
    
    merged = pd.merge(left, right, how="outer", left_on=left_on, right_on=right_on, suffixes=("", "_" + str(n)))
    
    left_rows = merged[left_on].notnull() # & merged[right_on].notnull()
    left = merged.loc[left_rows, [col for col in merged.columns if col != "in_response_to_tweet_id"]]
    
    right_rows = ~left_rows # merged[left_on].isnull() & merged[right_on].notnull()
    right = merged.loc[right_rows, ["tweet_id_" + str(n), "text_" + str(n), right_on]]
    right.columns = ["tweet_id", "text", right_on]
    
    n = n + 1
    if n > 5:
        break
    
    
    

In [44]:
first_tweets = tweets_csv[tweets_csv.in_response_to_tweet_id.isnull()].copy()
responses = tweets_csv[tweets_csv.in_response_to_tweet_id.notnull()].copy()

first_interactions = pd.merge(first_tweets, responses, how="left", left_on="tweet_id", right_on="in_response_to_tweet_id")
first_interactions = first_interactions[["tweet_id_x", "text_x", "text_y"]]
first_interactions.columns = ["id", "issue", "response"]

In [100]:
stop = set(stopwords.words('english'))
punct = "".join([p for p in string.punctuation if i not in ["-"]])
lemma = WordNetLemmatizer()

def get_cleaned_doc(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in punct)
    normalized = [lemma.lemmatize(word) for word in punc_free.split()]
    return normalized

documents = [get_cleaned_doc(doc) for doc in tweets_csv.text]


[['115712',
  'understand',
  'would',
  'like',
  'assist',
  'you',
  'would',
  'need',
  'get',
  'private',
  'secured',
  'link',
  'assist'],
 ['sprintcare', 'propose'],
 ['sprintcare',
  'sent',
  'several',
  'private',
  'message',
  'one',
  'responding',
  'usual'],
 ['115712',
  'please',
  'send',
  'u',
  'private',
  'message',
  'assist',
  'you',
  'click',
  '‘message’',
  'top',
  'profile'],
 ['sprintcare', 'did'],
 ['115712',
  'please',
  'send',
  'u',
  'private',
  'message',
  'gain',
  'detail',
  'account'],
 ['sprintcare', 'worst', 'customer', 'service'],
 ['115713',
  'saddening',
  'hear',
  'please',
  'shoot',
  'u',
  'dm',
  'look',
  'you',
  'kc'],
 ['sprintcare',
  'gonna',
  'magically',
  'change',
  'connectivity',
  'whole',
  'family',
  '🤥',
  '💯'],
 ['115713',
  'understand',
  'concern',
  'wed',
  'like',
  'please',
  'send',
  'u',
  'direct',
  'message',
  'assist',
  'you',
  'aa']]

In [101]:
dictionary = corpora.Dictionary(documents)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in documents]

KeyboardInterrupt: 

In [ ]:
doc_term_matrix

In [78]:
tweets_csv.text[:10].to_csv("Data/o.txt")

In [70]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'